In [1]:
# %%writefile query_test.hql

# use stackoverflow_; 
# show tables;

# describe posts_sample_external;

# select * from posts_sample_external limit 10;

In [2]:
# !hive -f query_test.hql

In [5]:
%%writefile hive_query_week2_task2_dml.hql

ADD JAR /opt/cloudera/parcels/CDH/lib/hive/lib/hive-contrib.jar;

USE stackoverflow_;

WITH year_tag_dist AS (
    SELECT year, exploded_tags, COUNT(*) AS cnt
    FROM posts
    --FROM posts_sample_external
    LATERAL VIEW explode(tags) tagsTable AS exploded_tags
    WHERE post_type_id = 1 AND year IN (2009, 2016)
    GROUP BY year, exploded_tags
)

SELECT concat_ws("\t", exploded_tags, string(rank_in_2016), string(rank_in_2009), string(cntags_in_2016), string(cntags_in_2009))
FROM (
    SELECT
        tags_in_2016.exploded_tags AS exploded_tags
        ,tags_in_2016.rank AS rank_in_2016
        ,tags_in_2009.rank AS rank_in_2009
        ,tags_in_2016.cnt AS cntags_in_2016
        ,tags_in_2009.cnt AS cntags_in_2009
    FROM (
        SELECT exploded_tags, cnt, RANK() OVER (ORDER BY cnt DESC) rank
        FROM year_tag_dist
        WHERE year = 2016
        LIMIT 10
    ) tags_in_2016
    LEFT OUTER JOIN (
        SELECT exploded_tags, cnt, RANK() OVER (ORDER BY cnt DESC) rank
        FROM year_tag_dist
        WHERE year = 2009
    ) tags_in_2009
    ON tags_in_2016.exploded_tags = tags_in_2009.exploded_tags
    ORDER BY rank_in_2016
) t_top;

Overwriting hive_query_week2_task2_dml.hql


In [6]:
!hive -f hive_query_week2_task2_dml.hql


Logging initialized using configuration in jar:file:/usr/local/apache-hive-2.3.6-bin/lib/hive-common-2.3.6.jar!/hive-log4j2.properties Async: true
Added [/opt/cloudera/parcels/CDH/lib/hive/lib/hive-contrib.jar] to class path
Added resources: [/opt/cloudera/parcels/CDH/lib/hive/lib/hive-contrib.jar]
OK
Time taken: 0.77 seconds
Query ID = jovyan_20201225181236_b8e551ee-e613-408b-adc7-ddcbd8a497c5
Total jobs = 8
Launching Job 1 out of 8
Number of reduce tasks not specified. Estimated from input data size: 1
In order to change the average load for a reducer (in bytes):
  set hive.exec.reducers.bytes.per.reducer=<number>
In order to limit the maximum number of reducers:
  set hive.exec.reducers.max=<number>
In order to set a constant number of reducers:
  set mapreduce.job.reduces=<number>
Starting Job = job_1608918779705_0001, Tracking URL = http://172.17.0.33:8088/proxy/application_1608918779705_0001/
Kill Command = /usr/local/hadoop/bin/hadoop job  -kill job_1608918779705_0001
Hadoop jo